Goal of analysis script is to pull in Med-PC files from Pavlovian conditioning experiment and process for Pavlovian Conditioned Approach (PCA) parameters

Med PC program for pavlovian conditioning has the following event_stamps (array E):
-  Event stamps ( array E )
-   1   = Rt lever press/NP
-   2   = Lt lever press/NP
-   5   = Pellet reinforcement delivery
-   6   = Food tray head entry
-   7   = Right cue light on
-   8   = Left cue light on
-   9   = Right cue light off
-   10  = Left cue ight off
-   11  = Magazine tray light on
-   12  = Magazine tray light off
-   13  = Liquid delivery turns on
-   14  = Liquid delivery turns off
-   15  = ITI starts
-   16  = ITI ends
-   18  = Right lever extends
-   19  = Left lever extends
-   20  = Right lever retracts
-   21  = Left lever retracts
-   22  = CS on
-   23  = CS off
-   100 = Session termination
-   999 = Place holder - no event

Import modules. MedMod is self-created module for processing Med PC files

In [1]:
import numpy as np
import pandas as pd
import re
import os
from scipy import stats
from tkinter import filedialog
from tkinter import *
import MedMod as mm


# visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Open dialog box and prompt user to select directory containing directories of behavioral sessions

In [3]:
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav


Use os module to create list of session directories

In [4]:
session_dirs = os.listdir(root.directory)
session_dirs

['Pav1', 'Pav2', 'Pav3', 'Pav4', 'Pav5', 'Pav6', 'Pav7']

Loop through session directory names to create a path for session directory

In [5]:
dir_paths = []
for directory in session_dirs:
    path = root.directory + '/' + directory
    dir_paths.append(path)

dir_paths

['C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav1',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav2',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav3',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav4',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav5',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav6',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav7']

Loop through session directorypaths to create a path for each session file contained in each directory

In [6]:
session_files = []
for directory in dir_paths:
    files = os.listdir(directory)
    for file in files:
        path = directory + '/' + file
        session_files.append(path)

print(len(session_files))
print(session_files[2])

140
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/Pavlovian/round 1 6.2018/Pav/Pav1/180618_269_Pav1


In [7]:
def analyze_PCA_from_med(file_path_name):
    #get file info
    date, animal_number, session = mm.get_file_info(file_path_name)
    #make array from file path
    array = mm.make_array_from_path(file_path_name)
    #get event times and event stamps
    es_et = mm.get_eandt(array, "E", "T")
    #get cs and iti start and end times
    cs_iti = mm.get_cs_and_iti(es_et, 22, 23, 15, 16)
    #Use dataframe of start and end times to select events during each CS and compute PCA parameters
    PCA_df = mm.PCA_analysis(cs_iti, es_et)
    PCA_df['Date'] = date
    PCA_df['Animal'] = int(animal_number)
    PCA_df['Session'] = session
    
    return PCA_df

In [8]:
PCA_session = pd.DataFrame()
for file in session_files:
    temp = analyze_PCA_from_med(file)
    PCA_session = pd.concat((PCA_session, temp), axis = 0)

In [9]:
PCA_session.head()

,lever_press,head_entries,lever_prob,head_prob,ave_lat_lever,ave_lat_head,response_bias,prob_diff,lat_score,PCA_score,Date,Animal,Session
0,4,1,0.12,0.04,4.146667,5.070,0.600000,0.08,0.092333,0.257444,180618,266,Pav1
0,6,1,0.20,0.04,5.916000,4.590,0.714286,0.16,-0.132600,0.247229,180618,268,Pav1
0,5,0,0.12,0.00,4.266667,0.000,1.000000,0.12,-0.426667,0.231111,180618,269,Pav1
0,6,6,0.20,0.16,3.494000,2.255,0.000000,0.04,-0.123900,-0.027967,180618,270,Pav1
0,9,4,0.20,0.08,3.558000,7.455,0.384615,0.12,0.389700,0.298105,180618,271,Pav1


In [10]:
#confirm that each animal has 7 sessions
PCA_session['Animal'].value_counts()

292    7
291    7
267    7
268    7
269    7
270    7
271    7
273    7
274    7
275    7
277    7
278    7
280    7
282    7
283    7
287    7
288    7
289    7
290    7
266    7
Name: Animal, dtype: int64

In [11]:
#read in animal groups, create dataframe
animal_info = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Analysis/Excel files/Med boxes/animals_round1.xlsx'
animal_info_data = pd.read_excel(animal_info)
animal_info_data

,Animal,Group
0,282,2
1,283,2
2,287,3
3,288,3
4,289,3
5,290,3
6,291,3
7,292,3
8,266,4
9,267,4


In [12]:
#use merge to add each animal's group number to the PCA_session dataframe
PCA_group = PCA_session.merge(animal_info_data, on='Animal')
PCA_group.head()

,lever_press,head_entries,lever_prob,head_prob,ave_lat_lever,ave_lat_head,response_bias,prob_diff,lat_score,PCA_score,Date,Animal,Session,Group
0,4,1,0.12,0.04,4.146667,5.070000,0.6,0.08,0.092333,0.257444,180618,266,Pav1,4
1,0,3,0.00,0.08,0.000000,3.210000,-1.0,-0.08,0.321000,-0.253000,180619,266,Pav2,4
2,2,6,0.08,0.08,7.910000,5.105000,-0.5,0.00,-0.280500,-0.260167,180620,266,Pav3,4
3,0,8,0.00,0.12,0.000000,4.640000,-1.0,-0.12,0.464000,-0.218667,180621,266,Pav4,4
4,0,17,0.00,0.36,0.000000,4.187778,-1.0,-0.36,0.418778,-0.313741,180626,266,Pav5,4


In [13]:
PCA_group.groupby(['Session', 'Group']).mean()

lever_press  head_entries  lever_prob  head_prob  ave_lat_lever  ave_lat_head  response_bias  prob_diff  lat_score  PCA_score      Animal
Session Group                                                                                                                                           
Pav1    2         2.000000      5.500000    0.060000   0.100000       5.287500      5.304167      -0.444444  -0.040000   0.001667  -0.160926  282.500000
        3         7.166667      2.833333    0.213333   0.060000       5.770222      2.368333       0.458333   0.153333  -0.340189   0.090493  289.500000
        4         5.333333      3.333333    0.153333   0.080000       3.985222      3.871667       0.349817   0.073333  -0.011356   0.137265  268.500000
        5         5.166667      2.333333    0.166667   0.053333       5.767352      4.491667       0.361111   0.113333  -0.127569   0.115625  276.166667
Pav2    2         6.000000      1.000000    0.160000   0.040000       5.722143      4.260000       0.416667   0.120000  -0.146214   0.130151  282.500000
        3         4.666667      4.333333    0.146667   0.120000       4.002262      5.847194      -0.140741   0.026667   0.184493   0.023473  289.500000
        4         1.333333      4.833333    0.040000   0.106667       3.678333      4.038750      -0.654762  -0.066667   0.036042  -0.228462  268.500000
        5         2.500000      7.833333    0.080000   0.166667       4.199583      3.720792      -0.336908  -0.086667  -0.047879  -0.157151  276.166667
Pav3    2         0.000000      2.000000    0.000000   0.060000       0.000000      5.155000      -1.000000  -0.060000   0.515500  -0.181500  282.500000
        3         5.500000      4.333333    0.126667   0.113333       5.518000      4.006306       0.018781   0.013333  -0.151169  -0.039685  289.500000
        4         1.833333      7.166667    0.066667   0.120000       6.273333      4.921389      -0.523148  -0.053333  -0.135194  -0.237225  268.500000
        5         1.666667      3.000000    0.060000   0.073333       6.524444      3.386667       0.027778  -0.013333  -0.313778  -0.099778  276.166667
Pav4    2         1.000000     13.500000    0.040000   0.260000       7.570000      4.284881      -0.853535  -0.220000  -0.328512  -0.467349  282.500000
        3         7.833333      9.500000    0.160000   0.240000       3.826481      5.014926      -0.286937  -0.080000   0.118844  -0.082697  289.500000
        4         2.000000      4.500000    0.046667   0.106667       4.142500      4.501250      -0.327778  -0.060000   0.035875  -0.117301  268.500000
        5         2.000000      6.666667    0.066667   0.166667       4.246667      5.038718      -0.455556  -0.100000   0.079205  -0.158783  276.166667
Pav5    2         0.500000      9.000000    0.020000   0.200000       0.655000      4.843000      -0.928571  -0.180000   0.418800  -0.229924  282.500000
        3        11.000000      7.166667    0.166667   0.166667       3.198362      4.220000      -0.028846   0.000000   0.102164   0.024439  289.500000
        4         2.666667      9.833333    0.093333   0.226667       5.067500      4.934352      -0.435450  -0.133333  -0.013315  -0.194033  268.500000
        5         2.166667      9.666667    0.080000   0.180000       5.061250      4.609139      -0.625198  -0.100000  -0.045211  -0.256803  276.166667
Pav6    2         1.000000      6.500000    0.040000   0.180000       3.790000      4.361786      -0.818182  -0.140000   0.057179  -0.300334  282.500000
        3         6.666667      4.833333    0.153333   0.093333       3.554937      3.608833       0.138158   0.060000   0.005390   0.067849  289.500000
        4         2.500000     18.000000    0.073333   0.260000       3.299722      4.438528      -0.588095  -0.186667   0.113881  -0.220294  268.500000
        5         1.500000     14.833333    0.033333   0.240000       1.751667      3.502273      -0.825746  -0.206667   0.175061  -0.285784  276.166667
Pav7    2         1.000000      6.500